In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import Request, urlopen
import urllib
import pandas as pd
from pandas import DataFrame as df

In [2]:
url = 'https://jayeuijunglee.github.io/website/html.htm'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

In [3]:
soup

<!DOCTYPE html>

<html>
<body>
<strong> ○천안시 공모사업 관리에 관한 조례안</strong>
<br/>
<strong> ○천안시 노동자 권리보호 및 복리증진에 관한 조례안</strong>
<br/>
<strong> ○천안시 비정규직 권리보호 및 지원에 관한 조례안</strong> <a href="https://www.google.com"> 맨처음으로</a>
<br/>
<strong>○의장 인치견</strong> 의사일정 제1항 천안시 공모사업 관리에 관한 조례안, 의사일정 제2항 천안시 노동자 권리보호 및 복리증진에 관한 조례안, 의사일정 제3항 천안시 대형유통기업 지역기여 권고 및 소상공인 보호 조례 일부개정조례안, 의사일정 제4항 천안시 비정규직 권리보호 및 지원에 관한 조례안. 이상 4건의 안건을 일괄 상정합니다. 
 <br/>
본 안건을 심사해 주신 경제산업위원회 김철환 부위원장께서는 심사결과를 보고하여 주시기 바랍니다. 
<br/>
<hr/>
<strong> ○유니세프 아동친화도시 추진 지방정부협의회 운영 규약 개정 동의안</strong> (시장제출)
<br/>
<strong>○의장 인치견</strong> 의사일정 제5항 유니세프 아동친화도시 추진 지방정부협의회 운영 규약 개정 동의안, 의사일정 제6항 대한민국 축구종합센터 도시개발사업 구역지정 및 개발계획 수립을 위한 의견청취. 이상 2건의 안건을 일괄 상정합니다. 
<br/>
<hr/>
<strong> ○천안시 대형유통기업 지역기여 권고 및 소상공인 보호 조례 일부개정조례안</strong>
<br/>
<strong> ○대한민국 축구종합센터 도시개발사업 구역지정 및 개발계획 수립을 위한 의견청취 </strong>
<br/>
본 안건을 심사하여 주신 복지문화위원회 김월영 부위원장께서는 심사결과를 보고하여 주시기 바랍니다. 
<br/>
<strong>○복지문화부위원장 김월영</strong> 복지문화위원회 김월영 부위원장입니다. 
 <br/>
본 위원회로 회부된 2개의 안건

# * Method 1: 안건 문자 배열 특징 활용
<pre>
문자 배열 특징 활용, agenda의 경우는 첫 칸을 띄워두고 있음
strong 태그 다 불러온 뒤 첫 칸을 띄워두고 있는 것을 안건으로 가져오고, 
첫 칸이 안 띄워져 있는 곳을 경계로 segement_number 구분

장점: 코드 굉장히 간단
단점: 만약에 안건의 첫 칸이 빈칸으로 되어 있지 않다면?? --> hr 태그의 위치 정보를 활용하는 코드가 필요
                                            --> hr 태그의 위치 정보 확인하는 코드가 조금 더 general 한 코드가 될 듯
</pre>

In [4]:
contents = soup.html.find_all("strong", text =True)

In [5]:
information = []
for i in range(len(list(contents))):
    information.append(contents[i].get_text())

In [6]:
list_numbering = []
list_contents = []
number = 1

for target in information:
    if target[0] == " ":
        list_numbering.append(number)
        list_contents.append(target)
    else:
        number += 1

In [7]:
test = pd.DataFrame({'segment_number':list_numbering, 'agenda_title': list_contents})

In [8]:
test

,segment_number,agenda_title
0,1,○천안시 공모사업 관리에 관한 조례안
1,1,○천안시 노동자 권리보호 및 복리증진에 관한 조례안
2,1,○천안시 비정규직 권리보호 및 지원에 관한 조례안
3,2,○유니세프 아동친화도시 추진 지방정부협의회 운영 규약 개정 동의안
4,3,○천안시 대형유통기업 지역기여 권고 및 소상공인 보호 조례 일부개정조례안
5,3,○대한민국 축구종합센터 도시개발사업 구역지정 및 개발계획 수립을 위한 의견청취


In [9]:
test.to_csv("test1.csv")

___

# * Method 2: hr 태그 위치 정보 확인해서 안건 구분

In [10]:
contents = soup.html.find_all("strong", text =True)
contents

[<strong> ○천안시 공모사업 관리에 관한 조례안</strong>,
 <strong> ○천안시 노동자 권리보호 및 복리증진에 관한 조례안</strong>,
 <strong> ○천안시 비정규직 권리보호 및 지원에 관한 조례안</strong>,
 <strong>○의장 인치견</strong>,
 <strong> ○유니세프 아동친화도시 추진 지방정부협의회 운영 규약 개정 동의안</strong>,
 <strong>○의장 인치견</strong>,
 <strong> ○천안시 대형유통기업 지역기여 권고 및 소상공인 보호 조례 일부개정조례안</strong>,
 <strong> ○대한민국 축구종합센터 도시개발사업 구역지정 및 개발계획 수립을 위한 의견청취 </strong>,
 <strong>○복지문화부위원장 김월영</strong>]

In [11]:
information = []                             # 첫번째에 블랭크가 있는 것만 안건으로 취급
for i in range(len(list(contents))):
    if contents[i].get_text()[0] == " ":
        information.append(contents[i].get_text())
        
information

[' ○천안시 공모사업 관리에 관한 조례안',
 ' ○천안시 노동자 권리보호 및 복리증진에 관한 조례안',
 ' ○천안시 비정규직 권리보호 및 지원에 관한 조례안',
 ' ○유니세프 아동친화도시 추진 지방정부협의회 운영 규약 개정 동의안',
 ' ○천안시 대형유통기업 지역기여 권고 및 소상공인 보호 조례 일부개정조례안',
 ' ○대한민국 축구종합센터 도시개발사업 구역지정 및 개발계획 수립을 위한 의견청취 ']

In [12]:
hr_soup = soup.find_all('hr')
agenda = [hr_soup[i].find_next().get_text() for i in range(len(list(hr_soup)))]  #두번째 세그먼트 부터 첫번째 안건 정보가 agenda에 담김
agenda

[' ○유니세프 아동친화도시 추진 지방정부협의회 운영 규약 개정 동의안',
 ' ○천안시 대형유통기업 지역기여 권고 및 소상공인 보호 조례 일부개정조례안']

In [13]:
list_numbering = []
number = 1
for target in information:
    try:
        if target == agenda[number-1]:
            number += 1
            list_numbering.append(number)
        else:
            list_numbering.append(number)
    
    except:
        list_numbering.append(number)

list_numbering


[1, 1, 1, 2, 3, 3]

In [14]:
test2 = pd.DataFrame({'segment_number':list_numbering, 'agenda_title': information})

In [15]:
test2

,segment_number,agenda_title
0,1,○천안시 공모사업 관리에 관한 조례안
1,1,○천안시 노동자 권리보호 및 복리증진에 관한 조례안
2,1,○천안시 비정규직 권리보호 및 지원에 관한 조례안
3,2,○유니세프 아동친화도시 추진 지방정부협의회 운영 규약 개정 동의안
4,3,○천안시 대형유통기업 지역기여 권고 및 소상공인 보호 조례 일부개정조례안
5,3,○대한민국 축구종합센터 도시개발사업 구역지정 및 개발계획 수립을 위한 의견청취


In [16]:
test2.to_csv("test2.csv")